In [18]:
#pip install firebase_admin
#pip install beautifulsoup4 

In [17]:
import firebase_admin
import FinanceDataReader as fdr
import datetime
import pandas as pd

from firebase_admin import credentials
from firebase_admin import db

from dateutil.relativedelta import relativedelta

In [3]:
#Firebase database 인증 및 앱 초기화
cred = credentials.Certificate('kb-app-23eb5-firebase-adminsdk-72lxa-1fddf52ea4.json')
firebase_admin.initialize_app(cred,{
    'databaseURL' : 'https://kb-app-23eb5-default-rtdb.firebaseio.com/'
})

In [52]:
today_default = datetime.date.today()
yesterday_default = today_default - datetime.timedelta(days=5)
tomorrow_default = today_default - datetime.timedelta(days=4)

yesterday_default = str(yesterday_default)
today_default = str(today_default)
tomorrow_default = str(tomorrow_default)

yesterday = yesterday_default.replace("-",'')
today = today_default.replace("-",'')
tomorrow = tomorrow_default.replace("-",'')

In [117]:
stocklist = fdr.StockListing('KRX') #나중에 stocklist 변경하기
stocks = pd.concat([stocklist.Name, stocklist.Symbol], axis = 1)

tic = "".join(list(stocks[stocks.Name=='삼성전자'].Symbol.values))
print(tic)
df = fdr.DataReader(tic,yesterday_default,yesterday_default)
print(df.Change.values)

005930
[0.00572246]


In [114]:
stocklist = fdr.StockListing('KRX') #나중에 stocklist 변경하기
stocks = pd.concat([stocklist.Name, stocklist.Symbol], axis = 1)

#dateIndex = today
#rank = pd.read_csv(f'./data/rank/{dateIndex}_rank.csv',encoding='utf-8-sig')
#companylist = list(rank.word)
#answer = pd.DataFrame({'word':companylist})

dateIndex = 20220325
companylist = ['삼성전자','NAVER','카카오','LG에너지솔루션','SK하이닉스','삼성바이오로직스','삼성SDI', '현대차', 'LG화학','기아']
answer = pd.DataFrame({'word':companylist})
changelst = []

for element in companylist:
    tic = "".join(list(stocks[stocks.Name==element].Symbol.values))
    df = fdr.DataReader(tic,yesterday_default,yesterday_default)
    #print(df.Change.values)
    if df.Change.values>0:
        changelst.append('up')
    elif df.Change.values==0:
        changelst.append('stay')
    else:
        changelst.append('down')

answer['change'] = changelst
answer.to_csv(f'./data/answer/{dateIndex}_answer.csv',encoding='utf-8-sig',index=False)

display(answer)
     

,word,change
0,삼성전자,up
1,NAVER,up
2,카카오,up
3,LG에너지솔루션,up
4,SK하이닉스,up
5,삼성바이오로직스,up
6,삼성SDI,up
7,현대차,up
8,LG화학,up
9,기아,up


## TodayStock Update

In [127]:
result = pd.read_csv(f'./data/result/{dateIndex}_result.csv',encoding='utf-8-sig')
max_score = result[result.score==result.score.max()]
min_score = result[result.score==result.score.min()]
if max_score.score.values[0]>0 : 
    goodstock = max_score.company.values[0] # 나중에 result company => word로 변환
else:
    goodstock = '아쉽지만 찾을 수 없습니다'
if min_score.score.values[0]<0:
    badstock = min_score.company.values[0] # 나중에 result company => word로 변환
else:
    badstock = '다행히 없습니다'
todaystockUpdate = {dateIndex:{'good':goodstock,'bad':badstock}}
#display(result)


ref = db.reference()
todaystock_ref = ref.child('todaystock')
todaystock_ref.update(todaystockUpdate)


## Ranking Update

In [161]:
dateIndex = 20220325
rank = pd.read_csv(f'./data/rank/{dateIndex}_rank.csv',encoding='utf-8-sig')

companyRank = {}
for i in range(len(rank.index)):
    companyRank[i] = rank.word[i]
rankUpdate = {dateIndex:companyRank}

ref = db.reference()
rank_ref = ref.child('ranking')
rank_ref.update(rankUpdate)

In [48]:
ref = db.reference()

overall_ref = ref.child('overall')
overall_ref.set({
    yesterday : '돌다리도 두드려 보고 건너라',
    today : '물에 빠진 사람 건져줬더니 보따리 내놓으라 한다',
    tomorrow : '백지장도 맞들면 낫다'   
})

todaystock_ref = ref.child('todaystock')
todaystock_ref.set({
    yesterday:
        {'good': '현대자동차',
         'bad' : '카카오'},
    today: {
        'good' : '삼성전자',
        'bad' : '스튜디오드래곤'
    },
    tomorrow:{
        'good' : '네이버',
        'bad' : '삼성화재'
    }
    
})

rank_ref = ref.child('ranking')
rank_ref.set({
    yesterday:
        {1: '현대자동차',
         2 : '카카오',
         3 : '회사3',
         4 : '회사4',
         5 : '회사5',
         6 : '회사6',
         7 : '회사7',
         8 : '회사8',
         9 : '회사9',
         10 : '회사10'},
    today: {
        1 : '삼성전자',
        2 : '스튜디오드래곤',
        3 : '회사3',
        4 : '회사4',
        5 : '회사5',
        6 : '회사6',
        7 : '회사7',
        8 : '회사8',
        9 : '회사9',
        10 : '회사10'
    },
    tomorrow:{
        1 : '네이버',
        2 : '삼성화재',
        3 : '회사3',
        4 : '회사4',
        5 : '회사5',
        6 : '회사6',
        7 : '회사7',
        8 : '회사8',
        9 : '회사9',
        10 : '회사10'
    }
    
})

checked_ref = ref.child('checked')
checked_ref.set({
    yesterday:{
        'service' : {
            1 : '⭕',
            2 : '⭕',
            3 : '⭕',
            4 : '⭕',
            5 : '⭕',
            6 : '⭕',
            7 : '⭕',
            8 : '⭕',
            9 : '⭕',
            10 : '❌'
            },
        'user': {
            1 : '❌',
            2 : '❌',
            3 : '⭕',
            4 : '⭕',
            5 : '⭕',
            6 : '⭕',
            7 : '⭕',
            8 : '⭕',
            9 : '⭕',
            10 : '⭕'
            }
        },
    today:{
        'service' : {
            1 : '⭕',
            2 : '⭕',
            3 : '⭕',
            4 : '⭕',
            5 : '⭕',
            6 : '⭕',
            7 : '⭕',
            8 : '⭕',
            9 : '⭕',
            10 : '⭕'
            },
        'user': {
            1 : '⭕',
            2 : '⭕',
            3 : '⭕',
            4 : '⭕',
            5 : '⭕',
            6 : '⭕',
            7 : '⭕',
            8 : '⭕',
            9 : '⭕',
            10 : '⭕'
            }
        },
    tomorrow:{
        'service' : {
            1 : '⭕',
            2 : '❌',
            3 : '❌',
            4 : '⭕',
            5 : '⭕',
            6 : '⭕',
            7 : '⭕',
            8 : '⭕',
            9 : '⭕',
            10 : '⭕'
            },
        'user': {
            1 : '❌',
            2 : '❌',
            3 : '❌',
            4 : '⭕',
            5 : '⭕',
            6 : '⭕',
            7 : '⭕',
            8 : '⭕',
            9 : '⭕',
            10 : '⭕'
            }
        }
})

### 채점결과